In [25]:
import os
import nltk
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import random
from nltk.corpus import stopwords
import operator
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler


stopwords = nltk.corpus.stopwords.words('english')
new_stopwords = ['/','(',')','{','}','@','|',';','\n','#','+','_','.',',','``',"''",':','-']
stopwords.extend(new_stopwords)

lemmatizer = nltk.stem.WordNetLemmatizer()

path = os.getcwd() + "\\bbc"
list_catagory_files = os.listdir(path)
print(list_catagory_files)
num_catagory = len(list_catagory_files)

path = os.getcwd() + "\\bbc"
list_catagory_files = os.listdir(path) # get bbc file
print(list_catagory_files)
num_catagory = len(list_catagory_files)

def text_label_list(cata_path):
    label = 0
    all_text_list = []
    all_label_list = []
    catagory_file_num = []
    
    for catagory in list_catagory_files:
        catagory_file = path + "\\" + catagory
        print(catagory_file)
        if catagory_file.endswith('.TXT'):
            continue
        else:
        
            file_num = len(os.listdir(catagory_file))
            catagory_file_num.append(file_num)
            print(file_num)
            print(catagory)
            for i in range(1,file_num + 1):
                all_label_list.append(label)
                j = "%03d" % i
                with open("\\".join([catagory_file, str(j) + ".txt"])) as fp:
                    lines = fp.read()
                    article = []
                    text_ = ""
                    text_ += lines
                    article.append(text_)    
                        
                        
                    all_text_list.append(article)
        label += 1
    return all_text_list, all_label_list, catagory_file_num




all_text_list_, all_label_list_, catagory_file_num = text_label_list(list_catagory_files)      
        
# get full dataset in order to split our dataset into training and test
dataset_full=[] 
for text, label in zip(all_text_list_, all_label_list_):
    dataset_full.append((text,label))
        

def get_train_test_split(dataset_full,ratio):
    pre_train_set=[]
    pre_test_set=[]
    
    size_dataset_full = len(dataset_full)
    test_size = int(round(size_dataset_full * ratio))
    list_test_indices=random.sample(range(size_dataset_full), test_size)
    
    for i,text in enumerate(dataset_full):
        if i in list_test_indices:
            pre_test_set.append(text)
        else:pre_train_set.append(text)
        
    return pre_train_set,pre_test_set
# get training set, test set
train_set, test_set = get_train_test_split(dataset_full,0.8)
print ("Size training set: "+str(len(train_set)))
print ("Size test set: "+str(len(test_set)))


# get training set, test set, development set
new_test_set, new_dev_set = get_train_test_split(test_set,0.5)

new_train_set=train_set
random.shuffle(new_train_set)
random.shuffle(new_dev_set)
random.shuffle(new_test_set)
print ("Size training set: "+str(len(new_train_set)))
print ("Size test set: "+str(len(new_test_set)))
print ("Size development set: "+str(len(new_dev_set)))



def get_list_tokens(text_list):
    paragraph = ",".join(text_list)
    sentence_split=nltk.tokenize.sent_tokenize(paragraph)
    
    list_tokens=[]
    for sentence in sentence_split:
        list_tokens_sentence=nltk.tokenize.word_tokenize(sentence)
        for token in list_tokens_sentence:
            list_tokens.append(lemmatizer.lemmatize(token).lower())
    return list_tokens

def get_vector_text(list_vocab,text_list):
    vector_text=np.zeros(len(list_vocab))
    paragraph = ",".join(text_list)
    list_tokens_string=get_list_tokens(paragraph)
    for i, word in enumerate(list_vocab):
        if word in list_tokens_string:
            vector_text[i]=list_tokens_string.count(word)
    return vector_text


def get_vocabulary(training_set, num_features): # Function to retrieve vocabulary
    dict_word_frequency={}
    for instance in training_set:
        sentence_tokens=get_list_tokens(instance[0])
        for word in sentence_tokens:
            if word in stopwords: continue
            if word not in dict_word_frequency: dict_word_frequency[word]=1
            else: dict_word_frequency[word]+=1
    sorted_list = sorted(dict_word_frequency.items(), key=operator.itemgetter(1), reverse=True)[:num_features]
    vocabulary=[]
    for word,frequency in sorted_list:
        vocabulary.append(word)
    return vocabulary   #word frequency 


def get_bigram_vocab(training_set):
    one_list = []
    article_tokens = []
    for instance in training_set:
        article = ','.join(instance[0])
        article_tokens.append(article)
    
    paragraph = ",".join(article_tokens)
    one_list.append(paragraph)
    bigram_vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2))
    text_all_vec = bigram_vectorizer.fit_transform(one_list)
    
    return bigram_vectorizer

def get_bigram_vec(training_set, vectorizer):
    vec = bigram_vectorizer.transform(training_set).toarray()
    return vec


def get_hash_vec(training_set):
    
    hashing_vectorizer = HashingVectorizer(n_features = 20)
    text_all_vec = hashing_vectorizer.transform(training_set).toarray()
    
    return text_all_vec
    

def chi_square_select(X_train,Y_train,num_features):
    fs_sentanalysis = SelectKBest(chi2, k=num_features).fit(X_train, Y_train)
    X_train_new = fs_sentanalysis.transform(X_train)
    return fs_sentanalysis, X_train_new

def append_list(list_0,list_1):
    vec_mul= np.append(list_0,list_1)
    return vec_mul

def minmax_scale(list_):
    
    list_ = list_.reshape(-1,1)
    
    scaler = MinMaxScaler()
    scaler.fit(list_)
    new_mul_fea = scaler.transform(list_)
    return new_mul_fea

def train_svm_classifier(training_set, vocabulary, chi_num): # Function for training our svm classifier
    X_train=[]
    Y_train=[]
    for instance in training_set:
        
        vector_fre = get_vector_text(vocabulary,instance[0])
        vector_hash = get_hash_vec(instance[0])
        vec_mul = append_list(vector_hash, vector_fre)
        vec_mul = minmax_scale(vec_mul)
        
        X_train.append(vec_mul)
        
        Y_train.append(instance[1])
    
    X_train = np.asarray(X_train)
    Y_train = np.asarray(Y_train)
    
    nsamples, nx, ny = X_train.shape
    d2_X_train = X_train.reshape((nsamples,nx*ny))
    
    fs_sentanalysis, X_train_chi_vector = chi_square_select(d2_X_train,Y_train,chi_num)
    
    
  # Finally, we train the SVM classifier 
    svm_clf=sklearn.svm.SVC(kernel="linear",gamma='auto')
    svm_clf.fit(X_train_chi_vector,Y_train)
    return svm_clf, fs_sentanalysis




vocabulary = get_vocabulary(new_train_set, 10)
svm_clf, fs_sentanalysis = train_svm_classifier(new_train_set, vocabulary,15)

['business', 'entertainment', 'politics', 'README.TXT', 'sport', 'tech']
['business', 'entertainment', 'politics', 'README.TXT', 'sport', 'tech']
C:\Users\c2008016\Desktop\Programs\coursework\bbc\business
510
business
C:\Users\c2008016\Desktop\Programs\coursework\bbc\entertainment
386
entertainment
C:\Users\c2008016\Desktop\Programs\coursework\bbc\politics
417
politics
C:\Users\c2008016\Desktop\Programs\coursework\bbc\README.TXT
C:\Users\c2008016\Desktop\Programs\coursework\bbc\sport
511
sport
C:\Users\c2008016\Desktop\Programs\coursework\bbc\tech
401
tech
Size training set: 445
Size test set: 1780
Size training set: 445
Size test set: 890
Size development set: 890


In [29]:
X_test=[]
Y_test=[]
for instance in new_test_set:   
    vector_fre = get_vector_text(vocabulary,instance[0])
    vector_hash = get_hash_vec(instance[0])
    vec_mul_test = append_list(vector_hash, vector_fre)
    vec_mul_test = minmax_scale(vec_mul_test)
    
    X_test.append(vec_mul_test)
    Y_test.append(instance[1])
X_test=np.asarray(X_test)
nsamples, nx, ny = X_test.shape
d2_X_test = X_test.reshape((nsamples,nx*ny))
Y_test_gold=np.asarray(Y_test)


from sklearn.metrics import classification_report

Y_text_predictions = svm_clf.predict(fs_sentanalysis.transform(d2_X_test))
print(Y_text_predictions)
print(classification_report(Y_test_gold, Y_text_predictions))

[2 2 0 3 3 2 2 4 0 2 3 3 1 3 3 3 3 0 3 3 3 1 0 3 1 2 3 4 4 4 3 2 2 4 0 3 3
 0 2 1 3 2 2 3 0 2 0 2 0 3 1 3 3 1 0 3 0 4 0 4 2 0 2 1 2 2 3 3 3 1 3 2 4 3
 4 0 0 4 0 3 0 0 2 1 1 1 3 3 1 3 1 4 3 3 3 4 0 4 0 3 0 4 2 3 1 3 0 0 4 0 3
 3 0 3 3 3 0 0 1 0 3 3 0 3 3 0 1 4 2 4 2 2 3 4 1 0 0 2 0 3 1 3 4 0 0 0 2 3
 0 3 0 0 3 3 3 3 0 0 3 2 0 1 3 4 4 2 1 3 0 4 2 2 0 0 1 3 1 3 3 0 2 1 4 1 2
 0 2 1 1 0 0 3 4 0 3 0 0 3 0 1 3 3 0 3 3 2 3 0 3 3 1 2 0 4 2 3 3 2 3 3 0 1
 0 3 4 3 1 4 2 0 4 3 1 0 0 4 1 4 2 3 2 2 1 3 0 3 3 4 3 2 3 3 3 3 4 1 2 3 0
 1 3 1 0 2 0 0 4 0 0 4 3 0 1 0 1 0 1 2 0 3 1 2 1 1 0 0 4 3 2 0 4 0 1 0 3 0
 2 1 0 4 2 1 4 3 2 3 3 0 3 1 3 0 1 0 3 4 3 2 0 1 3 4 2 0 0 0 3 1 0 2 0 3 3
 1 0 3 2 2 3 0 3 4 2 0 0 0 0 2 0 3 0 2 2 0 2 0 0 2 0 1 0 3 4 0 2 0 3 3 1 3
 2 4 0 1 1 3 0 3 4 4 3 1 4 0 3 0 1 0 1 0 2 3 0 4 2 4 3 0 1 1 3 2 2 4 3 2 4
 2 4 3 2 2 0 0 0 4 4 4 3 3 3 1 0 0 3 0 1 0 2 1 4 2 3 3 1 0 0 3 3 2 1 4 4 0
 2 4 1 2 3 0 4 1 0 2 3 4 4 3 0 2 2 3 4 2 4 1 1 2 0 1 2 3 3 2 1 3 1 0 2 0 1
 0 3 3 3 1 2 1 0 3 4 4 0 

In [ ]:
# def append_list(list_0,list_1):
#     vec_mul= np.append(list_0,list1)
#     return vec_mul
# print(vec_ha)